Question 6

In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from tobit_post import q, loglikelihood, starting_values

# Load the dataset
data = pd.read_csv("Data.csv")
y = data['y'].values
x = data['x'].values

# Add an intercept term to x
x = np.column_stack((np.ones_like(x), x))

# Get starting values for theta
theta0 = starting_values(y, x)
print("Starting Values:", theta0)

# Define the objective function for minimization
def objective_function(theta):
    return q(theta, y, x)

# Estimate parameters using optimization
result = minimize(objective_function, theta0, method='L-BFGS-B')
theta_hat = result.x
print("Estimated Parameters:", theta_hat)

# Compute log-likelihood at the estimated parameters
ll_hat = loglikelihood(theta_hat, y, x)
print("Log-Likelihood at MLE:", np.sum(ll_hat))

# Extract the parameter estimates
beta_hat = theta_hat[:-1]
sigma_hat = theta_hat[-1]

# Calculate standard errors
hessian_inv = result.hess_inv.todense()
standard_errors = np.sqrt(np.diag(hessian_inv))

# Calculate t-ratios
t_ratios = theta_hat / standard_errors

# Compile results into a table
results_table = pd.DataFrame({
    'Parameter': ['beta_0', 'beta_1', 'sigma'],
    'Estimate': theta_hat,
    'Std. Error': standard_errors,
    't Ratio': t_ratios
})
print("Results:")
print(results_table)

# Save results to CSV for review
results_table.to_csv("estimation_results.csv", index=False)

# Optional: Plot the predicted vs. observed values
import matplotlib.pyplot as plt
from tobit_post import predict

y_pred, _ = predict(theta_hat, x)
plt.scatter(y, y_pred, alpha=0.6)
plt.xlabel("Observed y")
plt.ylabel("Predicted y")
plt.title("Observed vs. Predicted Values")
plt.show()


Starting Values: [-0.60068275  0.57252733  0.87964049]


ValueError: The user-provided objective function must return a scalar value.